# Ensemble learning

Ensemble learning is een manier om meerdere machine learning algorithmes te gaan combineren om een beter resultaat te bekomen.
Een voorbeeld hiervan is Random Forest dat een ensemble is van een aantal decision trees. 
Een belangrijke opmerking hierbij is dat meerdere soorten machine learning algoritmes gecombineerd kunnen worden.
Er zijn hier drie bekende varianten van, namelijk:
* Stacking
* Bagging
* Boosting

Deze varianten gaan we nu 1 voor 1 behandelen.

# Stacking 

Een schets van hoe een stacking-ensemble werkt zie je hieronder.
Bij de stacking-methode wordt de volledige trainingsdata gebruikt om meerdere modellen te trainen.
Deze kunnen maar moeten niet van hetzelfde algoritme zijn en ook de hyperparameters kunnen verschillen.
Nadat al deze modellen getrained zijn, wordt er nog model getrained dat een selectie maakt van wat de uiteindelijke voorspelling is op basis van de uitkomst van alle getrainde modellen.
Dit tweede model kan dus evalueren wanneer welk model het meest correct is.
Het algoritme hiervoor kan vrij gekozen worden en kan zelfs een ander ensemble zijn.
Bij stacking is dus niet strikt majority voting van toepassing.

![ensemble](images\stacking.png)

Een [stacking ensemble](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html) kan als volgt geimplementeerd worden in sklearn:

In [30]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier

# load iris dataset, split in test en train, perform scaling on X_train en X_test
X,y = load_iris(return_X_y=True, as_frame=True)
#display(X)
#display(y)

# split in train test
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size =0.2)

# create StackingClassifier bestaande uit random forest, logistic regression, svm, knn
estimators = [("logReg", LogisticRegression(C=1)), 
              ("SVM", SVC()), 
              ("tree2", DecisionTreeClassifier()), 
              ("tree1", DecisionTreeClassifier())]

stacking_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

Column

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("stack", stacking_clf)
])

parameters = [{
    "scaler__with_std" : [True, False],
    "stack__tree1__max_depth" : [4,8,12],
    "stack__tree2__max_depth" : [3,6,9],
    "stack__final_estimator": [LogisticRegression(), SVC()]
}]

searcher = GridSearchCV(pipe, param_grid=parameters)

searcher.fit(X_train, y_train)

# print score van algemeen ensemble en van individuele parameters
print("Acc train data", searcher.score(X_train, y_train))
print("Acc test data", searcher.score(X_test, y_test))

print(searcher.best_params_)

#print(searcher.best_estimator_.named_steps["stack"].estimators_[0].score(X_test, y_test))
#print(searcher.best_estimator_.named_steps["stack"].estimators_[1].score(X_test, y_test))
#print(searcher.best_estimator_.named_steps["stack"].estimators_[2].score(X_test, y_test))
#print(searcher.best_estimator_.named_steps["stack"].estimators_[3].score(X_test, y_test))

Acc train data 0.9916666666666667
Acc test data 0.9
{'scaler__with_std': False, 'stack__final_estimator': LogisticRegression(), 'stack__tree1__max_depth': 4, 'stack__tree2__max_depth': 3}


# Bagging

De tweede methode bagging is ook gekend onder de naam bootstrap bagging.
Dit houdt in dat elk model slechts getrained wordt op een deel van de data (bootstrapped samples).
Deze observaties om elk model te trainen worden willekeurig gekozen met teruglegging.
Typisch wordt er ongeveer 60% van de totale data gebruikt om elk model te trainen.
Wanneer elk model getrained is wordt er een standaard majority voting toegepast om de uiteindelijke predictie te bekomen (bij regressie wordt er het gemiddelde genomen).

![bagging](images\bagging.png)

Een belangrijke opmerking is dat theoretisch gezien meerdere types van machine learning algoritmes gecombineerd kunnen worden maar dit dit niet rechtstreeks gaat met sklearn zonder zelf een ensemble te implementeren. 
Dit is ook niet nodig omdat zolang je ensemble groot genoeg is, je elke gewenste accuraatheid kan bereiken.
Een implementatie van de [bagging methode](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) ziet er als volgt uit:

In [17]:
# create bagging classifier
from sklearn.ensemble import BaggingClassifier
bag = BaggingClassifier(base_estimator=SVC(), n_estimators=10, max_samples=0.6)

# train en predict op de iris dataset hou tijd bij met %time of time.time()
%time bag.fit(X_train, y_train)
%time bag.score(X_test, y_test)

print(bag.score(X_test, y_test))

import time
start = time.time()
bag.fit(X_train, y_train)
time.time() - start


Wall time: 18 ms
Wall time: 4 ms
0.9333333333333333


0.017005205154418945

# Boosting

De derde variant van ensemble learning methoden is boosting.
Dit is een aanpassing van de bagging methode waar de bootstrapped samples niet meer willekeurig zijn.
Bij boosting worden de modellen sequentieel getrained en gevalideerd (met de trainingsdata diet niet in de bootstrapped sample zit). 
De classificaties die verkeerd waren bij deze validatie stap hebben een grotere kans om in de bootstrapped sample te zitten van het volgende model.
Omdat de uitkomst van elk model nodig is voor het volgende kan dit niet geparallelliseerd worden waardoor de trainingstijd snel kan oplopen.
Het voordeel echter van deze methode is dat de accuraatheid van het gecombineerde model hoger gaat zijn dan bij bagging.
De meest bekende implementatie van deze methode wordt AdaBoost genoemd.
Daarnaast wordt tegenwoordig ook XGBoost benoemd wat staat voor Extreme Gradient Boosting.
Het probleem is echter dat deze techniek niet standaard in sklearn staat.
Hiervoor moet een extra package toegevoegd worden, meer informatie hierover vind je [hier](https://towardsdatascience.com/getting-started-with-xgboost-in-scikit-learn-f69f5f470a97).
Deze techniek is een speciale variant van een random forest (er wordt gewerkt met meerdere decision trees).
Meer informatie over de api vind je [hier](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier).
De voordelen van xgboost zijn:
* Hoge accuraatheid
* Heel snelle uitvoering door parallellisatie
* Flexibel algoritme door keuse van optimalisatie
* Kan omgaan met missing data
* Voert autmatisch pruning uit om overfitting tegen te gaan
* Ingebouwde cross-validatie

![boosting](images\boosting.png)

 Een implementatie van de [boosting methode](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) ziet er als volgt uit:

In [18]:
# create adaboost classifier
from sklearn.ensemble import AdaBoostClassifier
boost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())

# train en predict op de iris dataset hou tijd bij met %time of time.time()
%time boost.fit(X_train, y_train)
%time boost.score(X_test, y_test)

print(boost.score(X_test, y_test))

import time
start = time.time()
boost.fit(X_train, y_train)
time.time() - start

Wall time: 2 ms
Wall time: 0 ns
0.9666666666666667


0.0029942989349365234

Meer informatie over alle mogelijke ensemble-methoden in sklearn vind je [hier](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble).

# XGBoost

Gradient Boosting is een ML-techniek die iteratief modellen toevoegt aan een ensemble.
Er wordt begonnen met een eenvoudig model dat zeer naief is.
De voorspellingen van dit model worden gebruikt om een kost-functie te berekenen.
Deze functie wordt dan gebruikt voor een nieuw model te trainen dat toegevoegd aan het ensemble.
De parameters van dit nieuwe model worden zo gekozen dat de kost-functie verminderd.
Met dit uitgebreide ensemble worden opnieuw voorspelling gedaan die het dan terug gebruikt worden om een kostfunctie te berekenen.

De belangrijkste parameters van deze techniek zijn:
* n_estimators: Geeft het maximum aantal bomen in het ensemble weer. Een lage waarde kan leiden tot underfitting, een grote tot overfitting.
* learning rate: Dit kan gebruikt zorden om de impact van extra bomen te verkleinen om overfitting tegen te gaan. Lagere waarden gaan normaal een hogere accuraatheid maar ook een hogere trainingstijd opleveren.
* early stopping: Deze parameter moet meegegeven worden bij het fitten en kan gebruikt worden om overfitting tegen te gaan. Indien de validation score niet afneemt gedurende dit aantal rondes, dan stopt de fit-functie omdat er overfitting gedetecteerd wordt.
* eval_set: Deze parameter is een tuple van de validatieset dat gebruikt kan worden voor het early-stopping te testen. Deze moet dus met early-stopping gecombineerd worden.
* n_jobs: aantal cores dat kan gebruikt worden voor de training


Meer informatie over de parameters van deze techniek vind je [hier](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn).

In [ ]:
# install xgboost als niet gekend
!pip install xgboost

In [23]:
# create xgboost classifier
from xgboost import XGBClassifier
xgbooster = XGBClassifier(n_estimators=10000, learning_rate=0.1)

# train en predict op de iris dataset hou tijd bij met %time of time.time()
xgbooster.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], verbose=False)

xgbooster.score(X_test, y_test)

# probleem met gridsearch door de parameters in de fit() 
# -> early stopping en eval_set proberen kan niet geautomatiseerd worden
# early stopping rounds in nieuwere versie in constructor maar de eval_set nog niet

[11:58:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\jens.baetens3\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.9666666666666667